In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import owlready2 as owl
from owlready2 import *
owlready2.reasoning.JAVA_MEMORY = 20000

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

torch.manual_seed(33)

def negative_sampler_concept(self, data):

        with torch.no_grad():

            # Review this code
            
            # Sort the real batch by concept number and individual number
            print(f'data: {data}')
            print(f'data shape {data.shape}')

            sorted_concepts_idcs = torch.argsort(data[:,0])
            sorted_concepts_data = data[sorted_concepts_idcs]
            sorted_individual_idcs = torch.argsort(sorted_concepts_data[:,1] +
                                                   sorted_concepts_data[:,0] *
                                                   sorted_concepts_data.size(0))
            sorted_data_final = sorted_concepts_data[sorted_individual_idcs]

            print(f'sorted real batch conflict samples{sorted_data_final}')
            
            # Generate negative candidates and sort the corrupted batch

            neg_sampled_candidates = torch.randint(0, self.individual_embedding_dict.weight.shape[0], (data.shape[0],))
            corrupted_batch = torch.cat((data[:,0].unsqueeze(1), neg_sampled_candidates.unsqueeze(1)), dim=1)
            sorted_neg_concepts_idcs = sorted_concepts_idcs # Reutilize the previous sorting operation
            sorted_neg_concepts_batch = corrupted_batch[sorted_neg_concepts_idcs]
            sorted_corrupted_individual_idcs = torch.argsort(sorted_neg_concepts_batch[:,1] +
                                                             sorted_neg_concepts_batch[:,0] *
                                                             sorted_neg_concepts_batch.size(0))
            sorted_corrupted_batch_final = sorted_neg_concepts_batch[sorted_corrupted_individual_idcs]

            print(f'sorted corrupted conflict samples{sorted_corrupted_batch_final}')

            # While there are identical samples in both real and corrupted batches, generate more corrupted individuals

            # We check whether any assertion in our negatively sampled individuals is in the batch
            
            counter = 0
            MAX_ITER = 50000

            negsamp_checker = sorted_data_final[:,1] == sorted_corrupted_batch_final[:,1]

            while torch.any(negsamp_checker) and counter != MAX_ITER:
        
                conflict_idcs = torch.where(negsamp_checker == True)[0]
                for idx in conflict_idcs:
                    sorted_corrupted_batch_final[idx][1] = torch.randint(0, self.individual_embedding_dict.weight.shape[0], (1,)).item()

                counter += 1

            if counter == MAX_ITER:
                print('loop limit reached')
                #print(f'neg_sampchecker TRUE: {negsamp_checker}')
                where_conflict = torch.where(negsamp_checker)[0]

            # Return only the individuals, discarding concepts
            return sorted_corrupted_batch_final[:,1]

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import owlready2 as owl
from owlready2 import *
owlready2.reasoning.JAVA_MEMORY = 200000

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

#dir = '/Users/victorlacerda/Documents/VSCode/ELHFaithfulness/NormalizedOntologies/family_ontology.owl'
dir = '/Users/victorlacerda/Desktop/family_ontology.owl'

RESTRICT_LANGUAGE = False # If True, the language is restricted to simpler TBox axioms on the left-hand side of the rules

'''
Class for creating entities to
populate the creation of the
canonical models.
The .name attribute is used to
create a single representation
for concepts like A and B / 
B and A, as they are the same.
'''

class CanonicalModelElements:

    concept_names = {}
    concept_intersections = {}
    concept_restrictions = {}
    all_concepts = {}

    def __init__(self, concept):
        self.concept = concept
        self.name = self.get_name()
        self.get_element_dict()

    def get_name(self):
        
        if type(self.concept) == ThingClass:
            return self.concept.name

        elif type(self.concept) == Restriction:
            return 'exists_' + self.concept.property.name + '.' + self.concept.value.name
        
        else:
            return 'And_' + ''.join(sorted(self.concept.Classes[0].name + self.concept.Classes[1].name)) # The name is sorted to avoid that (e.g) (A \and B) and (B \and A) are treated as different concepts
        
    def get_element_dict(self):

        if type(self.concept) == ThingClass:
            CanonicalModelElements.concept_names[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == Restriction:
            CanonicalModelElements.concept_restrictions[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

        elif type(self.concept) == And:
            CanonicalModelElements.concept_intersections[self.name] = self
            CanonicalModelElements.all_concepts[self.name] = self

def get_canonical_model_elements(concept_names_iter, role_names_iter, ontology, restrict_language = False):
    
    onto = ontology
    top = owl.Thing
    #bottom = owl.Nothing

    CanonicalModelElements(top)
    #CanonicalModelElements(bottom)

    for concept_name in concept_names_iter:
        
        CanonicalModelElements(concept_name)

        if restrict_language == False:

            for concept_name2 in concept_names_iter:
        
                with onto:
                    gca = GeneralClassAxiom(concept_name & concept_name2)
                    gca.is_a.append(concept_name & concept_name2)
            
                CanonicalModelElements(gca.left_side)

    print('')
    print('===========================================================================================================')
    print('All Concept Names and Concept Intersections have been preprocessed for the creation of the canonical model.')
    print('===========================================================================================================')

    concept_names_iter.append(top)
    #concept_names_iter.append(bottom)

    if restrict_language == False:

        for role_name in role_names_iter:
            for concept_name in concept_names_iter:
                with onto:
                    gca = GeneralClassAxiom(role_name.some(concept_name))
                    gca.is_a.append(role_name.some(concept_name))

                CanonicalModelElements(gca.left_side)
    
    else:

        for role_name in role_names_iter:
            with onto:
                gca = GeneralClassAxiom(role_name.some(owl.Thing))
                gca.is_a.append(role_name.some(owl.Thing))

                CanonicalModelElements(gca.left_side)
            
    print('')
    print('All restrictions have been preprocessed for the creation of the canonical model.')


'''
The main class for creating the canonical model for the ontology.

The canonical model is stored in dictionaries available as class variables 'concept_canonical_interpretation'
and 'role_canonical_interpretation'. 

Args:
    concept_names_dict: a dictionary stored in the CanonicalModelElement class.
    concept_intersection_dict: a dictionary stored in the CanonicalModelElement class.
    concept_restrictions_dict: a dictionary stored in the CanonicalModelElement class.
    all_concepts_dict: a dictionary stored in the CanonicalModelElement class.
    role_names_iter (list): a list containing all role names in the loaded ontology.
'''



"\nThe main class for creating the canonical model for the ontology.\n\nThe canonical model is stored in dictionaries available as class variables 'concept_canonical_interpretation'\nand 'role_canonical_interpretation'. \n\nArgs:\n    concept_names_dict: a dictionary stored in the CanonicalModelElement class.\n    concept_intersection_dict: a dictionary stored in the CanonicalModelElement class.\n    concept_restrictions_dict: a dictionary stored in the CanonicalModelElement class.\n    all_concepts_dict: a dictionary stored in the CanonicalModelElement class.\n    role_names_iter (list): a list containing all role names in the loaded ontology.\n"

In [3]:
class CanonicalModel:

    concept_canonical_interpretation = {}
    role_canonical_interpretation = {}

    def __init__(self, concept_names_dict, concept_intersections_dict, concept_restrictions_dict, all_concepts_dict, role_names_iter):
        
        self.domain = all_concepts_dict
        self.concept_names_dict = concept_names_dict
        self.concept_restrictions_dict = concept_restrictions_dict
        self.concept_intersections_dict = concept_intersections_dict

        self.role_names_iter = role_names_iter

        self.concept_canonical_interp = self.get_concept_name_caninterp() # These are only used to build the concept_canonical_interpretation and role_canonical_interpretation class attributes
        self.role_canonical_interp = self.get_role_name_caninterp()       # The functions do not return anything, they just update their corresponding class variables 

    def get_concept_name_caninterp(self):

        # The variable concept is a string containing the name of an element of the domain of the canonical model
        # The key to the concept_names_dict variable corresponds to concept.name
        # This name can be used to access the concept in owlready2's format

        for concept in self.concept_names_dict.keys():

            CanonicalModel.concept_canonical_interpretation[concept] = []
            superclasses = self.domain[concept].concept.ancestors(include_self=True, include_constructs=True) # The self.domain[concept] is used to access the CanonicalModelElements type of object,
                                                                                                               # and the attribute .concept is used to access the concept in owlready2 format                                                            
            for superclass in superclasses:

                if type(superclass) == ThingClass:
                    CanonicalModel.concept_canonical_interpretation[concept].append(superclass.name)

                elif type(superclass) == Restriction:
                    CanonicalModel.concept_canonical_interpretation[concept].append('exists_' + superclass.property.name + '.' + superclass.value.name)

                elif type(superclass) == And:
                    if 'And_' + ''.join(sorted(superclass.Classes[0].name + superclass.Classes[1].name)) in CanonicalModel.concept_canonical_interpretation[concept]:
                        pass
                    else:
                        CanonicalModel.concept_canonical_interpretation[concept].append('And_' + ''.join(sorted(superclass.Classes[0].name + superclass.Classes[1].name)))

            
    def get_role_name_caninterp(self):

        # Initialize the dictionary storing the canonical interpretation of roles

        for role_name in self.role_names_iter:

            role_name_str = role_name.name # Accesses the property type object's name as a string
            CanonicalModel.role_canonical_interpretation[role_name_str] = []

        # First case from Definition 10
                                
        for role_name in self.role_names_iter:

            superroles = role_name.ancestors(include_self=True)
            print(superroles)

            for superrole in superroles:
                for restriction_name, restriction_concept in self.concept_restrictions_dict.items():

                    if superrole == restriction_concept.concept.property:
                        concept_name_str = restriction_concept.concept.value.name
                        pair = (restriction_name, concept_name_str)
                        CanonicalModel.role_canonical_interpretation[role_name.name].append(pair)
                        
        
        # Second case from Definition 10

        for restriction_name in self.concept_restrictions_dict.keys(): # Where restriction_name denotes an \exists r.B type of concept 'exists_' + self.concept.property.name + '.' + self.concept.value.name

            #print(f'Restriction name init for loop: {restriction_name}')
            restriction_concept = self.concept_restrictions_dict[restriction_name].concept
            c_B = self.concept_restrictions_dict[restriction_name].concept.value.name
            #print(f'c_B: {c_B}\n')

            superclasses = restriction_concept.ancestors(include_self=True, include_constructs=False)

            for superclass in superclasses:

                super_superclasses = superclass.ancestors(include_self=True, include_constructs=True)

                for super_superclass in super_superclasses:
                    if type(super_superclass) == ThingClass and super_superclass:
                        c_D = super_superclass.name
                        CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))

                    elif type(super_superclass) == Restriction:
                        c_D = 'exists_' + super_superclass.property.name + '.' + super_superclass.value.name
                        CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))

                    elif type(super_superclass) == And:
                        c_D = 'And_' + ''.join(sorted(super_superclass.Classes[0].name + super_superclass.Classes[1].name))
                        CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))
                    
        
            if role_name_str in restriction_name:

                #print(f'It is true that {role_name_str} is in {restriction_name}.')
                    
                superclasses = self.domain[restriction_name].concept.ancestors(include_self=True, include_constructs=False) # Include_constructs is turned to false due to the definition of canonical model

                #print(f'These are the superclasses of the restriction_name {restriction_name}:" {superclasses}')

                for superclass in superclasses:
                    super_superclasses = superclass.ancestors(include_self=True, include_constructs=True)

                    for super_superclass in super_superclasses:

                        if type(super_superclass) == ThingClass:
                            c_D = super_superclass.name
                            CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))

                        elif type(super_superclass) == Restriction:
                            c_D = 'exists_' + super_superclass.property.name + '.' + super_superclass.value.name
                            CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))

                        elif type(super_superclass) == And:
                            c_D = 'And_' + ''.join(sorted(super_superclass.Classes[0].name + super_superclass.Classes[1].name))
                            CanonicalModel.role_canonical_interpretation[role_name_str].append((c_D, c_B))

'''
Main function for creating the canonical model.

    Args:
        onto_dir (str): a string pointing to the directory where the ontology is stored.

    Returns:
        canmodel (CanonicalModel): returns a variable containing the canonical model. 
        
        Attention: the interpretations of concept names and role names can also be accessed via class variables
        from the CanonicalModel class.
'''

def create_canonical_model(onto_dir, restrict_language_flag):

    onto = get_ontology(onto_dir)
    onto = onto.load()

    individuals_iter = list(onto.individuals())
    gcas_iter = list(onto.general_class_axioms()) # Attention: this will not work unless the generator is converted into a list
    concept_names_iter = list(onto.classes())
    role_names_iter = list(onto.properties())

    get_canonical_model_elements(concept_names_iter, role_names_iter, onto, restrict_language_flag)

    print('============================================================================')
    print('Starting to reason.\n')

    with onto:
        sync_reasoner()

    gcas_iter = list(onto.general_class_axioms()) # Attention: this will not work unless the generator is converted into a list
    concept_names_iter = list(onto.classes())
    role_names_iter = list(onto.properties())
    individuals_iter = list(onto.individuals())

    print('')
    print('============================================================================')
    print('Done reasoning. Creating the canonical model.')
    canmodel = CanonicalModel(CanonicalModelElements.concept_names, CanonicalModelElements.concept_intersections, CanonicalModelElements.concept_restrictions, CanonicalModelElements.all_concepts, role_names_iter)
    print('============================================================================\n')
    print('Concluded creating canonical model.')

    return canmodel

# Instantiates the canonical model

canmodel = create_canonical_model(dir, RESTRICT_LANGUAGE)


All Concept Names and Concept Intersections have been preprocessed for the creation of the canonical model.

All restrictions have been preprocessed for the creation of the canonical model.
Starting to reason.



* Owlready2 * Running HermiT...
    java -Xmx200000M -cp /opt/homebrew/Caskroom/miniforge/base/envs/kgenv/lib/python3.11/site-packages/owlready2/hermit:/opt/homebrew/Caskroom/miniforge/base/envs/kgenv/lib/python3.11/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/wg/g5861gcs6k5d3rbq_rncztjw0000gn/T/tmpt9njl0iu



Done reasoning. Creating the canonical model.
{prop.P22.some(Class.Father)}
{prop.P22.some(Class.Mother)}
{prop.P22.some(Class.Spouse)}
{prop.P22.some(Class.Sibling)}
{prop.P22.some(Class.Child)}
{prop.P22.some(entity.Q5)}
{prop.P22.some(Class.Female)}
{prop.P22.some(Class.Male)}
{prop.P22.some(owl.Thing)}
{prop.P25.some(Class.Father)}
{prop.P25.some(Class.Mother)}
{prop.P25.some(Class.Spouse)}
{prop.P25.some(Class.Sibling)}
{prop.P25.some(Class.Child)}
{prop.P25.some(entity.Q5)}
{prop.P25.some(Class.Female)}
{prop.P25.some(Class.Male)}
{prop.P25.some(owl.Thing)}
{prop.P26.some(Class.Father)}
{prop.P26.some(Class.Mother)}
{prop.P26.some(Class.Spouse)}
{prop.P26.some(Class.Sibling)}
{prop.P26.some(Class.Child)}
{prop.P26.some(entity.Q5)}
{prop.P26.some(Class.Female)}
{prop.P26.some(Class.Male)}
{prop.P26.some(owl.Thing)}
{prop.P3373.some(Class.Father)}
{prop.P3373.some(Class.Mother)}
{prop.P3373.some(Class.Spouse)}
{prop.P3373.some(Class.Sibling)}
{prop.P3373.some(Class.Child)}
{prop.P

* Owlready2 * HermiT took 0.5342342853546143 seconds
* Owlready * Reparenting entity.Q16019673: {owl.Thing} => {Class.Father}
* Owlready * Reparenting entity.Q2397531: {owl.Thing} => {Class.Child, Class.Father}
* Owlready * Reparenting entity.Q57209: {owl.Thing} => {Class.Sibling}
* Owlready * Reparenting entity.Q829669: {owl.Thing} => {Class.Child}
* Owlready * Reparenting entity.Q11090991: {owl.Thing} => {Class.Sibling}
* Owlready * Reparenting entity.Q10323203: {owl.Thing} => {Class.Sibling}
* Owlready * Reparenting entity.Q2840038: {owl.Thing} => {Class.Child}
* Owlready * Reparenting entity.Q60610: {owl.Thing} => {Class.Child}
* Owlready * Reparenting entity.Q213716: {owl.Thing} => {Class.Sibling}
* Owlready * Reparenting entity.Q312110: {owl.Thing} => {Class.Father, Class.Child, Class.Spouse, Class.Sibling}
* Owlready * Reparenting entity.Q22876077: {owl.Thing} => {Class.Child}
* Owlready * Reparenting entity.Q2446902: {owl.Thing} => {Class.Child, Class.Father}
* Owlready * Repar